![alt text](<https://www.et-gw.eu/images/et-new-logo.png>)

# XIII ET Symposium: Hackathon Session


### GWFish Tutorial: a brief guide for your first steps in using GWFish

## Settings for Google Colab

In [ ]:
#! git clone https://github.com/janosch314/GWFish
#! cd GWFish
#! pip install .
#! pip install lalsuite

**Note 1**: in Google Colab remember to **restart the kernel runtime** after installation

**Note 2**: GWFish uses LALSimulation, so the package needs to be installed separately from GWFish

## Import packages

In [ ]:
import GWFish.modules as gw

from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import corner
import numpy as np
import pandas as pd
import json
from astropy.cosmology import Planck18

# Single Event Analysis: GW170817-like

## Initialize GWFish

Here we are using the **GW180817** BNS event as it would have been seen with a triangular shape ET located in Sardinia.

* We need to specify the population type (BNS or BBH), then the detectors and the network
* The choice of the waveform approximant can be done from all the availbale waveforms from **LALSimulation**
* All the detectors characteristics are specified in the **detectors.yaml** file
* For a more realistic analysis we can include the **duty cycle** of the detectors using `duty_cycle = True`

In [ ]:
population = 'BNS' # or 'BBH'
detectors = ['ET']
networks = '[[0]]'

detectors_ids = detectors
networks_ids = json.loads(networks)
ConfigDet = 'GWFish/detectors.yaml'
duty_cycle = False

waveform_model = 'IMRPhenomD_NRTidalv2'
waveform_class = gw.waveforms.LALFD_Waveform

### Injections

We can analyze single events or entire populations (see below). Here consider the GW170817 event. 

The parameters need to passed as a **DataFrame** with the following nomenclature:
- `mass_1`: primary mass of the binary in Msol (in source frame)
- `mass_2`: secondary mass of the binary in Msol (in source frame)
- `redshift`: the redshift of the merger
- `luminosity_distance`:  the luminosity distance of the merger in Mpc
- `theta_jn`: the inclination angle between the line of observation and the perpendicular to the binary plane in rad
- `dec`: declination angle in rad
- `ra`: right ascension in rad
- `psi`: the polarization angle in rad
- `phase`: the initial phase of the merger in rad
- `geocent_time`: merger time as GPS time
- `a_1`: dimensionless spin parameter of primary component
- `a_2`: dimensionless spin parameter of secondary component
- `lambda_1`: tidal polarizabilty of primary component
- `lambda_2`: tidal polarizabilty of secondary component

In [ ]:
z = np.array([0.00980])

parameters = {
    'mass_1': np.array([1.46]), 
    'mass_2': np.array([1.27]), 
    'redshift': z,
    'luminosity_distance': Planck18.luminosity_distance(z).value,
    'theta_jn': np.array([2.545065595974997]),
    'ra': np.array([3.4461599999999994]), # phi in gwfast
    'dec': np.array([-0.4080839999999999]), # pi/2 - theta
    'psi': np.array([0.]),
    'phase': np.array([0.]),
    'geocent_time': np.array([1187008882.4])}
parameters = pd.DataFrame(parameters)
print(parameters)

### Initialize Network

If we want to compute the Fisher matrix we need to specify the parameters in the detector network settigs. Moreover, we need to specify what is the minimum network SNR for Fisher analysis (the fiducial value is set to SNR = 8.)

In [ ]:
threshold_SNR = np.array([0., 8.])
fisher_parameters = ['mass_1', 'mass_2', 'luminosity_distance', 'theta_jn', 'dec','ra',
                     'psi']
network = gw.detection.Network(detectors_ids, detection_SNR=threshold_SNR, parameters=parameters,
                               fisher_parameters=fisher_parameters, config=ConfigDet)


### Calculate SNR

In [ ]:
networkSNR_sq = 0
parameter_values = parameters.iloc[0]

#for d in np.arange(len(network.detectors)):
for d in np.arange(1):
    data_params = {
    'frequencyvector': network.detectors[d].frequencyvector,
    'f_ref': 50.
    }
    
    waveform_obj = waveform_class(waveform_model, parameter_values, data_params)
    wave = waveform_obj()
    t_of_f = waveform_obj.t_of_f

    signal = gw.detection.projection(parameter_values, network.detectors[d], wave, t_of_f)

    SNRs = gw.detection.SNR(network.detectors[d], signal, duty_cycle=duty_cycle)
    print('Single SNR value:', SNRs)
    networkSNR_sq += np.sum(SNRs ** 2)
    network.detectors[d].SNR[0] = np.sqrt(np.sum(SNRs ** 2))
    
network.SNR = np.sqrt(networkSNR_sq)
print('The network SNR of the event is ', network.SNR)

### Plot Signal

In [ ]:
h_plus, h_cross = signal[:, 0], signal[:, 1]
ampl = np.sqrt(np.abs(h_plus)**2. + np.abs(h_cross)**2.)

psd_et_D = np.loadtxt('GWFish/detector_psd/ET_psd.txt', usecols=[0,1])
frequencyvector = network.detectors[d].frequencyvector[:, 0]

fig, ax = plt.subplots(figsize=(12,6))

ax.plot(frequencyvector, 2.*np.sqrt(frequencyvector)*ampl, 
        linewidth=2., label='%s' %(waveform_model))
ax.plot(psd_et_D[:,0],np.sqrt(psd_et_D[:,1]), linewidth=2.0, label='ET-D ASD')

ax.set_xlim(1.,frequencyvector[-1])
ax.set_ylim(1e-25, 1e-20)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('f [Hz]', fontsize=15)
ax.set_ylabel(r'$2 \ |\tilde{h}| \ \sqrt{f}\ [Hz^{-1/2}]$', fontsize=15)
plt.grid(linestyle='dotted', linewidth='0.6', which='both')
ax.legend(loc='upper right', fontsize=15, ncol=1, fancybox=True)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
conv_to_hours = 3600
ax.plot(frequencyvector, (t_of_f - parameters['geocent_time'].iloc[0]) / conv_to_hours, linewidth=2.)

ax.set_xlim(1.,frequencyvector[-1])
ax.set_xscale('log')
ax.set_xlabel('f [Hz]', fontsize=15)
ax.set_ylabel(r'time [hours]', fontsize=15)
plt.grid(linestyle='dotted', linewidth='0.6', which='both')

plt.show()

## Fisher matrix calculation and error analysis

In [ ]:
# Calculate Fisher matrix
for d in np.arange(len(network.detectors)):
    network.detectors[d].fisher_matrix[0, :, :] = \
                    gw.fishermatrix.FisherMatrix(waveform_model, parameter_values, 
                                                 fisher_parameters, network.detectors[d], 
                                                 waveform_class=waveform_class).fm

gw.fishermatrix.analyzeFisherErrors(network, parameters, fisher_parameters, population, networks_ids)

In [ ]:
errors = pd.read_csv('Errors_ET_%s_SNR%s.txt'%(population, threshold_SNR[1]), delimiter = ' ')
print(errors.iloc[0])

fishers = np.load('Fishers_ET_%s_SNR%s.npy' %(population, threshold_SNR[1]))
fisher_matrix = fishers[0, :, :]
print('fisher matrix', fisher_matrix)

inv_fishers = np.load('Inv_Fishers_ET_%s_SNR%s.npy' %(population, threshold_SNR[1]))
cov_matrix = inv_fishers[0, :, :]
print('covarinace matrix', cov_matrix)


prod = np.dot(fisher_matrix, cov_matrix)
npar = len(fisher_parameters)
for i in range(npar):
    for j in range(i + 1, npar):
        if(prod[i][j] > 1e-10):
            print('Inversion problem')

In [ ]:
errors = pd.read_csv('Errors_ET_%s_SNR%s.txt'%(population, threshold_SNR[1]), delimiter = ' ')
sky_loc_conv = 4.6 * (180. / np.pi)**2
print('Sky localization (in deg2) =', sky_loc_conv * errors['err_sky_location'].iloc[0])

## Corner plot

In [ ]:
CORNER_KWARGS = dict(
    bins = 50, # number of bins for histograms
    smooth = 0.99, # smooths out contours. 
    plot_datapoints = True, # choose if you want datapoints
    label_kwargs = dict(fontsize = 12), # font size for labels
    show_titles = True, #choose if you want titles on top of densities.
    title_kwargs = dict(fontsize = 12), # font size for title
    plot_density = False,
    title_quantiles = [0.16, 0.5, 0.84],  # add quantiles to plot densities for 1d hist
    levels = (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.)), # 1, 2 and 3 sigma contours for 2d plots
    fill_contours = True, #decide if you want to fill the contours
    max_n_ticks = 2, # set a limit to ticks in the x-y axes.
    title_fmt=".3f"
    )
corner_lbs = [r'$m_1$ $[M_{\odot}]$', '$m_2$ $[M_{\odot}]$', '$D_l$ [Mpc]',
                '$\iota$ [rad]', '$DEC$ [rad]', '$RA$ [rad]', '$\Psi$ [rad]']
mean_values = [parameters['mass_1'].iloc[0], parameters['mass_2'].iloc[0], 
               parameters['luminosity_distance'].iloc[0],
               parameters['theta_jn'].iloc[0], parameters['dec'].iloc[0], 
               parameters['ra'].iloc[0], parameters['psi'].iloc[0]]

# Sample from a multi-variate gaussian with the given covariance matrix and injected mean values
samples = np.random.multivariate_normal(mean_values, cov_matrix, int(1e6))
fig = corner.corner(samples, labels = corner_lbs, truths = mean_values, truth_color = 'red',
                    **CORNER_KWARGS)
plt.show()

#

# BNS Population Analysis

* The `threshold_SNR` specifies in the first entry what is the minimum SNR for a detector to be included in the network, while the second entry is the lower threshold for the network SNR so that the Fisher matrix of the event is calculated
* If we want to calculate the uncertainties on the injected parameters values we need to specify `calculate_errors = True` and list the parameters to include in the Fisher analysis `fisher_parameters = ['...']`
* For a more realistic analysis we can include the duty cycle of the detectors using `duty_cycle = True`

In [ ]:
population = 'BNS' # or 'BBH'
detectors = ['ET']
networks = '[[0]]'

detectors_ids = detectors
networks_ids = json.loads(networks)
ConfigDet = 'GWFish/detectors.yaml'

threshold_SNR = np.array([0., 8.])
calculate_errors = True
fisher_parameters = ['mass_1', 'mass_2', 'luminosity_distance', 'theta_jn', 'dec','ra',
                     'psi', 'geocent_time', 'phase', 'lambda_1', 'lambda_2']
duty_cycle = True

np.random.seed(0)

waveform_model = 'IMRPhenomD_NRTidalv2'
waveform_class = gw.waveforms.LALFD_Waveform

In [ ]:
ns = 100 # or 50 or 100 or 200
parameters = pd.read_csv('injections/injections_my_BNS_pop_file_ns_%s.txt' %(ns), delimiter = ' ')
print(parameters.keys())
print(len(parameters))

network = gw.detection.Network(detectors_ids, detection_SNR=threshold_SNR, parameters=parameters,
                                   fisher_parameters=fisher_parameters, config=ConfigDet)

In [ ]:
for k in tqdm(np.arange(len(parameters))):
    parameter_values = parameters.iloc[k]
    
    networkSNR_sq = 0
    for d in np.arange(len(network.detectors)):
        data_params = {
            'frequencyvector': network.detectors[d].frequencyvector,
            'f_ref': 50.
        }
        waveform_obj = waveform_class(waveform_model, parameter_values, data_params)
        wave = waveform_obj()
        t_of_f = waveform_obj.t_of_f

        signal = gw.detection.projection(parameter_values, network.detectors[d], wave, t_of_f)

        SNRs = gw.detection.SNR(network.detectors[d], signal, duty_cycle=duty_cycle)
        networkSNR_sq += np.sum(SNRs ** 2)
        network.detectors[d].SNR[k] = np.sqrt(np.sum(SNRs ** 2))

        if calculate_errors:
            network.detectors[d].fisher_matrix[k, :, :] = \
                gw.fishermatrix.FisherMatrix(waveform_model, parameter_values, fisher_parameters, network.detectors[d], waveform_class=waveform_class).fm

    network.SNR[k] = np.sqrt(networkSNR_sq)

gw.detection.analyzeDetections(network, parameters, population, networks_ids)

if calculate_errors:
    gw.fishermatrix.analyzeFisherErrors(network, parameters, fisher_parameters, population, networks_ids)


In [ ]:
#signals = pd.read_csv('Signals_%s.txt' %(population), delimiter = ' ')
signals = pd.read_csv('Signals_%s_ns_1000.txt' %(population), delimiter = ' ')
print(signals.keys())

## Plots of injected data

In [ ]:
plt.hist(signals['ET_SNR'], bins = 25, alpha = 0.5)
plt.xlabel('SNR')
plt.ylabel('number of events')
plt.grid(linestyle='dotted', linewidth='0.6', which='both')

plt.show()

In [ ]:
plt.hist(signals['mass_1'], bins = 25, label = 'mass_1', alpha= 0.5)
plt.hist(signals['mass_2'], bins = 25, label = 'mass_2', alpha= 0.5)
plt.legend()
plt.xlabel('mass [$M_{\odot}$]')
plt.ylabel('number of events')
plt.grid(which = 'both', color = 'lightgray', alpha = 0.5, linewidth = 0.5)
plt.show()

In [ ]:
cm = matplotlib.colormaps.get_cmap('binary')
sc = plt.scatter(signals['dec'], signals['ra'], c = signals['ET_SNR'], cmap = cm)
plt.xlabel('dec [rad]')
plt.ylabel('ra [rad]')
plt.colorbar(sc, label = 'SNR')
plt.grid(which = 'both', color = 'lightgray', alpha = 0.5, linewidth = 0.5)
plt.show()

In [ ]:
#errors = pd.read_csv('Errors_ET_%s_SNR%s.txt'%(population, threshold_SNR[1]), delimiter = ' ')
errors = pd.read_csv('Errors_ET_%s_SNR%s_ns_1000.txt'%(population, threshold_SNR[1]), delimiter = ' ')
print(errors.keys())

In [ ]:
# Cut the results to max sky_loc = whole_sky
whole_sky = 4 * np.pi
errors = errors.loc[errors['err_sky_location'] <= whole_sky]

sky_loc_conv = 4.6 * (180. / np.pi)**2
sc = plt.scatter(errors['err_luminosity_distance']/errors['luminosity_distance'],
                 sky_loc_conv * errors['err_sky_location'],
                 c = errors['network_SNR'], cmap = cm)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$\Delta d_L/d_L$')
plt.ylabel('$\Delta \Omega_{90}$')
plt.colorbar(sc, label = 'SNR')
plt.grid(which = 'both', color = 'lightgray', alpha = 0.5, linewidth = 0.5)
plt.show()

In [ ]:
plt.hist(errors['err_theta_jn']/errors['theta_jn'], bins = np.logspace(-2, 2, 50), alpha = 0.5)
plt.xscale('log')
plt.xlabel('$\Delta \iota/\iota$')
plt.ylabel('number of events')
plt.grid(which = 'both', color = 'lightgray', alpha = 0.5, linewidth = 0.5)
plt.show()

In [ ]:
plt.hist(errors['err_luminosity_distance']/errors['luminosity_distance'], bins = np.logspace(-2, 2, 50), alpha = 0.5)
plt.xscale('log')
plt.xlabel('$\Delta d_L/d_L$')
plt.ylabel('number of events')
plt.grid(which = 'both', color = 'lightgray', alpha = 0.5, linewidth = 0.5)
plt.show()